This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = ''

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import collections
import requests
import datetime

In [3]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
date = datetime.datetime(2020,7,10)
url = "https://www.quandl.com/api/v3/datasets/FSE/AFX_X?start_date="+date.strftime("%Y-%m-%d")+"&end_date="+date.strftime("%Y-%m-%d")+"&api_key="+API_KEY
response = requests.get(url)
# into the JSON structure that will be returned
json_format = response.json()

In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure

json_format

{'dataset': {'id': 10095370,
  'dataset_code': 'AFX_X',
  'database_code': 'FSE',
  'name': 'Carl Zeiss Meditec (AFX_X)',
  'description': 'Stock Prices for Carl Zeiss Meditec (2020-06-26) from the Frankfurt Stock Exchange.<br><br>Trading System: Xetra<br><br>ISIN: DE0005313704',
  'refreshed_at': '2020-07-10T22:16:32.795Z',
  'newest_available_date': '2020-07-10',
  'oldest_available_date': '2000-06-07',
  'column_names': ['Date',
   'Open',
   'High',
   'Low',
   'Close',
   'Change',
   'Traded Volume',
   'Turnover',
   'Last Price of the Day',
   'Daily Traded Units',
   'Daily Turnover'],
  'frequency': 'daily',
  'type': 'Time Series',
  'premium': False,
  'limit': None,
  'transform': None,
  'column_index': None,
  'start_date': '2020-07-10',
  'end_date': '2020-07-10',
  'data': [['2020-07-10',
    89.25,
    91.55,
    88.9,
    89.9,
    None,
    93540.0,
    8453243.25,
    None,
    None,
    None]],
  'collapse': None,
  'order': None,
  'database_id': 6129}}

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [5]:
# Now we will declare the start and end date of the year 2017 to get all the data of that corresponding year
date_start = datetime.datetime(2017,1,1).strftime("%Y-%m-%d")
date_end = datetime.datetime(2017,12,31).strftime("%Y-%m-%d")
print("Date Start "+date_start+" | Date End "+date_end)

Date Start 2017-01-01 | Date End 2017-12-31


In [6]:
# Now we will pass the start_date, end_date and API_KEY to get the trading data of Zeiss in the year 2017
new_url = "https://www.quandl.com/api/v3/datasets/FSE/AFX_X?start_date="+date_start+"&end_date="+date_end+"&api_key="+API_KEY
response = requests.get(new_url)
# We will call .json() method of the requests module which automaically returns us the data in Json format.
data_year = response.json()
#Json is automatically parsed as a dictionary in Python
data_year['dataset']['data'][0]

['2017-12-29',
 51.76,
 51.94,
 51.45,
 51.76,
 None,
 34640.0,
 1792304.0,
 None,
 None,
 None]

In [7]:
# Now we will create four lists that contain only the Open, High, low and close price of each individual day in the dataset
# Note we are using is not None to filter out any null values if present
open_values = [x[1] for x in data_year['dataset']['data'] if x[1] is not None]
high_values = [x[2] for x in data_year['dataset']['data'] if x[2] is not None]
low_values = [x[3] for x in data_year['dataset']['data'] if x[3] is not None]
close_values = [x[4] for x in data_year['dataset']['data'] if x[4] is not None]

In [8]:
# Now we will use min() and max() to find the Minimum and Maximum Opening Price of the entire Dataset
minimum_value = min(open_values)
maximum_value = max(open_values)
print('Highest Opening Price is : {}. Lowest Opening Price is : {}'.format(maximum_value,minimum_value))

Highest Opening Price is : 53.11. Lowest Opening Price is : 34.0


In [9]:
# Now we need to find the largest change in a single based on high and low values.
# We will high_values and low_values lists for this purpose
largest_change_in_one_day = 0
length = len(high_values)
for i in range(length):
    #Calculate the difference between High and Low values and save them to a variable
    change = high_values[i] - low_values[i]
    if largest_change_in_one_day < change:
        largest_change_in_one_day = change

print('Largest change in a single day is : %.2f'%(largest_change_in_one_day))

Largest change in a single day is : 2.81


In [10]:
# Now we need to find the largest change between two days based on closing values.
# We will close_values lists for this purpose
largest_change_between_two_days = 0
length = len(close_values)
for i in range(length):
    prev_value = 0
    if(i > 0):
        prev_value = close_values[i-1]
    else: 
        prev_value = close_values[i]
    current_value = close_values[i]
    difference = current_value - prev_value
    if largest_change_between_two_days < difference:
        largest_change_between_two_days = change

print('Largest change between two days is : %.2f'%(largest_change_between_two_days))

Largest change between two days is : 0.95


In [11]:
# Calculate the average trading volume by adding all the traded volumes in the dataset and divide it by 365
trading_volume = [x[6] for x in data_year['dataset']['data'] if x[6] is not None]
average_daily_trading_volume = sum(trading_volume)/365
print('Average Daily Trading Volume : %.2f'%(average_daily_trading_volume))

Average Daily Trading Volume : 62264.95


In [12]:
# To get the median trading volume get the midpoint index and find the value for that index
median_trading_volume = trading_volume[round(len(trading_volume)/2) - 1]
print('Median trading volume this year is %.2f'%(median_trading_volume))

Median trading volume this year is 134965.00
